In [29]:
import numpy as np
import numpy.linalg
import scipy.linalg

## Linsolve Exact with LR


## Linsolve Least Squares with QR

In [30]:
def back_substitution(U, y):
    n = U.shape[1]
    x = np.zeros_like(y, dtype=np.double)
    x[-1] = y[-1] / U[-1, -1]
    for i in range(n - 2, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i:], x[i:])) / U[i, i]

    return x

In [31]:
x = np.array([5000, 6000])
y = np.array([100, 110])

# create matrix for model function (e.g.): y = p1 + p2 * x
A = np.vstack((np.ones_like(x), x)).T

num_param = A.shape[1]
q, r = scipy.linalg.qr(A) # Q*R*p = y
parameters = back_substitution(r[0:num_param], (q.T @ y)[0:num_param]) # solve for: R*p = Q.T * y
p1, p2 = parameters
print(parameters)

[5.e+01 1.e-02]


## Linsolve Least Squares with Cholesky

In [32]:
t = np.array([0.00, 0.03, 0.05, 0.08, 0.10])
u = np.array([5.0, 2.94, 1.73, 1.01, 0.6])

S = np.array([np.ones_like(t), -t/100]).T

A = S.T@S
b = S.T@(np.log(u))
L = numpy.linalg.cholesky(A)
p = scipy.linalg.solve_triangular(L, b, lower=True) / np.diag(L)
print(p)

[5.47018722e-01 2.11390200e+03]
